In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import json
import time

import pandas as pd
import numpy as np

C:\Users\romai\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
def get_list_of_url(tag,year,month,test=False):
    browser = webdriver.Chrome()
    browser.get(f"https://medium.com/tag/{tag}/archive/{year}/{month}")
    
    print(f"Scrapping https://medium.com/tag/{tag}/archive/{year}/{month}...")
    
    list_of_articles = []
    articles = []

    last_height = browser.execute_script("return document.body.scrollHeight")
    
    if test == True:
        itemTargetCount = 10
    else:
        itemTargetCount = 1000
    
    flag_bottom_page = 0 #to avoid break before scrapping articles
    
    while itemTargetCount > len(list_of_articles) and flag_bottom_page == 0:

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2)

        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height==last_height:
            flag_bottom_page = 1
        last_height = new_height

        articles = browser.find_elements(By.TAG_NAME, 'article')

    flag=0

    for article in articles:
        flag+=1
        print("Scrapping article n°"+str(flag))

        # get the authors
        try:
            authors=article.find_element(By.TAG_NAME, 'p').text
        except:
            authors= None

        # get the title
        try:
            title=article.find_element(By.TAG_NAME, 'h2').text
        except:
            title=None

        # get the url
        try:
            url=article.find_elements(By.TAG_NAME, 'a')[3].get_attribute('href')
        except:
            url=None

        # get the claps
        try:
            claps=article.find_element(By.TAG_NAME, "button").text
            if "." in claps:
                claps=claps.replace(".","")
                claps=claps.replace("K","00")
            else:
                claps=claps.replace("K","000")
            claps=int(claps)
        except:
            claps= 0

        # get the respond                    ! ! !   doesn't work quite well    ! ! !
        try:
            respond=article.find_element(By.TAG_NAME, "span").text
            respond=int(respond)
        except:
            respond= 0

        dictionnary_of_article = {"Tag":tag, "year":year ,"month": month, "claps":claps, "respond":respond, "title": title, "authors": authors, "link": url }

        list_of_articles.append(dictionnary_of_article) 

    #browser.quit() # close Chrome

    json.dump(list_of_articles, open("items.json", "w"))

    print(f"end of get_list_of_url() https://medium.com/tag/{tag}/archive/{year}/{month} :)")

    import pandas as pd
    df = pd.DataFrame.from_dict(list_of_articles)
    return df

In [5]:
def get_details(df, test=False):

    list_of_article_urls = df["link"].tolist()
    print("Number of articles to be scrapped:")
    print(len(list_of_article_urls))
    print("Beginning...")

    flag=0

    list_of_article_datas = []
    
    
    if test == True:
        list_of_article_urls = list_of_article_urls[:10]
    else:
        pass
    
    for article in list_of_article_urls:

        browser = webdriver.Chrome()

        flag+=1
        print("Scrapping article n°"+str(flag))
        print(article)

        try:
            browser.get(article)

            time.sleep(2)

            flag_2=0

            # get the authors
            try:
                authors=browser.find_elements(By.TAG_NAME, "a")[10].text
                """
                # developing loop for writing the right request
                
                flag_author=0
                print(len(authors))
                for e in authors:
                    print("_________________________________________________trying ->"+str(flag_author))
                    try:
                        print(e.text)
                    except:
                        print(" -> not this one: "+str(flag_author))
                    flag_author+=1
                """
            except:
                authors= None

            # get the title
            try:
                #print("trying title")
                title=browser.find_elements(By.TAG_NAME, 'h1')[0].text
                #print(len(title))
                #print(title.text)
            except:
                title=None

            # get the catch_phrase
            try:
                catch_phrase=browser.find_elements(By.TAG_NAME, 'h2')[0].text
            except:
                catch_phrase=None

            # get the text
            try:
                text=""
                body=browser.find_element(By.TAG_NAME, 'article')
                paragrapher=body.find_elements(By.TAG_NAME, 'p')
                for p in paragrapher[7:]:
                    text+=" "+p.text
            except:
                text=None

            # get the url
            try:
                url=article
            except:
                url=None

            # get the categories
            try:
                categories_chunck=body.find_elements(By.XPATH, "/html/body/div[1]/div/div[3]/div[2]/div[5]/div")
                categories=categories_chunck[0].text
                categories=categories.replace("\n",", ")

            except:
                categories= None

            # get the claps
            try:
                claps=browser.find_element(By.TAG_NAME, "footer").find_elements(By.TAG_NAME, "button")[1].text
                if "." in claps:
                    claps=claps.replace(".","")
                    claps=claps.replace("K","00")
                else:
                    claps=claps.replace("K","000")
                claps=int(claps)
            except:
                claps= None

            # get the respond
            try:
                respond=browser.find_element(By.TAG_NAME, "footer").find_elements(By.TAG_NAME, "button")[2].text
                respond=int(respond)
            except:
                respond= None

            # get the date
            try:
                date=browser.find_elements(By.TAG_NAME, "span")[18].text
            except:
                date=None

            dictionnary_of_data = {"date": date, "categories": categories, "title": title,
                                   "Claps":claps, "Respond":respond,
                                   "authors": authors,
                                   "catch_phrase":catch_phrase,"text": text, "link": url }

            list_of_article_datas.append(dictionnary_of_data) 
            """

            """

            browser.quit() # close Chrome
        except:
            print("xxxxxxxxxxxxxxxxxxxxxxxx Failed to scrap article n°"+str(flag))
            browser.quit() # close Chrome
    
    print("end of get_details()")
    
    import pandas as pd
    df_data = pd.DataFrame.from_dict(list_of_article_datas)
    return df_data

In [6]:
# scraping the main page to get index of articles

month=["1","2"]
year=["2024"]
tag=['conversational-ai',
 'deep-learning',
 'large-language-models',
 'machine-learning',
 'nlp',
 'voice-assistant',
 #'blockchain',
 #'bitcoin',
 #'cryptocurrency',
 #'decentralized-finance',
 #'ethereum',
 #'nft',
 #'web3',
 'data-science',
 'analytics',
 'data-engineering',
 'data-visualization',
 'database-design',
 'sql']

df_list = []

for t in tag:
    for y in year:
        for m in month:
            print(f"https://medium.com/tag/{t}/archive/{y}/{m}")
            df_temp = get_list_of_url(t,y,m,test=False)
            df_list.append(df_temp)

https://medium.com/tag/conversational-ai/archive/2024/1
Scrapping https://medium.com/tag/conversational-ai/archive/2024/1...
Scrapping article n°1
Scrapping article n°2
Scrapping article n°3
Scrapping article n°4
Scrapping article n°5
Scrapping article n°6
Scrapping article n°7
Scrapping article n°8
Scrapping article n°9
Scrapping article n°10
Scrapping article n°11
Scrapping article n°12
Scrapping article n°13
Scrapping article n°14
Scrapping article n°15
Scrapping article n°16
Scrapping article n°17
Scrapping article n°18
Scrapping article n°19
Scrapping article n°20
Scrapping article n°21
Scrapping article n°22
Scrapping article n°23
Scrapping article n°24
Scrapping article n°25
Scrapping article n°26
Scrapping article n°27
Scrapping article n°28
Scrapping article n°29
Scrapping article n°30
Scrapping article n°31
Scrapping article n°32
Scrapping article n°33
Scrapping article n°34
Scrapping article n°35
Scrapping article n°36
Scrapping article n°37
Scrapping article n°38
Scrapping 

Scrapping article n°141
Scrapping article n°142
Scrapping article n°143
Scrapping article n°144
Scrapping article n°145
Scrapping article n°146
Scrapping article n°147
Scrapping article n°148
Scrapping article n°149
Scrapping article n°150
Scrapping article n°151
Scrapping article n°152
Scrapping article n°153
Scrapping article n°154
Scrapping article n°155
Scrapping article n°156
Scrapping article n°157
Scrapping article n°158
Scrapping article n°159
Scrapping article n°160
Scrapping article n°161
Scrapping article n°162
Scrapping article n°163
Scrapping article n°164
Scrapping article n°165
Scrapping article n°166
Scrapping article n°167
Scrapping article n°168
Scrapping article n°169
Scrapping article n°170
Scrapping article n°171
Scrapping article n°172
Scrapping article n°173
Scrapping article n°174
Scrapping article n°175
Scrapping article n°176
Scrapping article n°177
Scrapping article n°178
Scrapping article n°179
Scrapping article n°180
Scrapping article n°181
Scrapping articl

Scrapping article n°483
Scrapping article n°484
Scrapping article n°485
Scrapping article n°486
Scrapping article n°487
Scrapping article n°488
Scrapping article n°489
Scrapping article n°490
Scrapping article n°491
Scrapping article n°492
Scrapping article n°493
Scrapping article n°494
Scrapping article n°495
Scrapping article n°496
Scrapping article n°497
Scrapping article n°498
Scrapping article n°499
Scrapping article n°500
Scrapping article n°501
Scrapping article n°502
Scrapping article n°503
Scrapping article n°504
Scrapping article n°505
Scrapping article n°506
Scrapping article n°507
Scrapping article n°508
Scrapping article n°509
Scrapping article n°510
Scrapping article n°511
Scrapping article n°512
Scrapping article n°513
Scrapping article n°514
Scrapping article n°515
Scrapping article n°516
Scrapping article n°517
Scrapping article n°518
Scrapping article n°519
Scrapping article n°520
Scrapping article n°521
Scrapping article n°522
Scrapping article n°523
Scrapping articl

Scrapping article n°827
Scrapping article n°828
Scrapping article n°829
Scrapping article n°830
Scrapping article n°831
Scrapping article n°832
Scrapping article n°833
Scrapping article n°834
Scrapping article n°835
Scrapping article n°836
Scrapping article n°837
Scrapping article n°838
Scrapping article n°839
Scrapping article n°840
Scrapping article n°841
Scrapping article n°842
Scrapping article n°843
Scrapping article n°844
Scrapping article n°845
Scrapping article n°846
Scrapping article n°847
Scrapping article n°848
Scrapping article n°849
Scrapping article n°850
Scrapping article n°851
Scrapping article n°852
Scrapping article n°853
Scrapping article n°854
Scrapping article n°855
Scrapping article n°856
Scrapping article n°857
Scrapping article n°858
Scrapping article n°859
Scrapping article n°860
Scrapping article n°861
Scrapping article n°862
Scrapping article n°863
Scrapping article n°864
Scrapping article n°865
Scrapping article n°866
Scrapping article n°867
Scrapping articl

Scrapping article n°1165
Scrapping article n°1166
Scrapping article n°1167
Scrapping article n°1168
Scrapping article n°1169
Scrapping article n°1170
Scrapping article n°1171
Scrapping article n°1172
Scrapping article n°1173
Scrapping article n°1174
Scrapping article n°1175
Scrapping article n°1176
Scrapping article n°1177
Scrapping article n°1178
Scrapping article n°1179
Scrapping article n°1180
Scrapping article n°1181
Scrapping article n°1182
Scrapping article n°1183
Scrapping article n°1184
Scrapping article n°1185
Scrapping article n°1186
Scrapping article n°1187
Scrapping article n°1188
Scrapping article n°1189
Scrapping article n°1190
Scrapping article n°1191
Scrapping article n°1192
Scrapping article n°1193
Scrapping article n°1194
Scrapping article n°1195
Scrapping article n°1196
Scrapping article n°1197
Scrapping article n°1198
Scrapping article n°1199
Scrapping article n°1200
Scrapping article n°1201
Scrapping article n°1202
Scrapping article n°1203
Scrapping article n°1204


...

Scrapping article n°609
Scrapping article n°610
Scrapping article n°611
Scrapping article n°612
Scrapping article n°613
Scrapping article n°614
Scrapping article n°615
Scrapping article n°616
Scrapping article n°617
Scrapping article n°618
Scrapping article n°619
Scrapping article n°620
Scrapping article n°621
Scrapping article n°622
Scrapping article n°623
Scrapping article n°624
Scrapping article n°625
Scrapping article n°626
Scrapping article n°627
Scrapping article n°628
Scrapping article n°629
Scrapping article n°630
Scrapping article n°631
Scrapping article n°632
Scrapping article n°633
Scrapping article n°634
Scrapping article n°635
Scrapping article n°636
Scrapping article n°637
Scrapping article n°638
Scrapping article n°639
Scrapping article n°640
Scrapping article n°641
Scrapping article n°642
Scrapping article n°643
Scrapping article n°644
Scrapping article n°645
Scrapping article n°646
Scrapping article n°647
Scrapping article n°648
Scrapping article n°649
Scrapping articl

Scrapping article n°952
Scrapping article n°953
Scrapping article n°954
Scrapping article n°955
Scrapping article n°956
Scrapping article n°957
Scrapping article n°958
Scrapping article n°959
Scrapping article n°960
Scrapping article n°961
Scrapping article n°962
Scrapping article n°963
end of get_list_of_url() https://medium.com/tag/data-visualization/archive/2024/2 :)
https://medium.com/tag/database-design/archive/2024/1
Scrapping https://medium.com/tag/database-design/archive/2024/1...
Scrapping article n°1
Scrapping article n°2
Scrapping article n°3
Scrapping article n°4
Scrapping article n°5
Scrapping article n°6
Scrapping article n°7
Scrapping article n°8
Scrapping article n°9
Scrapping article n°10
Scrapping article n°11
Scrapping article n°12
Scrapping article n°13
Scrapping article n°14
Scrapping article n°15
Scrapping article n°16
Scrapping article n°17
Scrapping article n°18
Scrapping article n°19
Scrapping article n°20
Scrapping article n°21
Scrapping article n°22
Scrapping

Scrapping article n°242
Scrapping article n°243
Scrapping article n°244
Scrapping article n°245
Scrapping article n°246
Scrapping article n°247
Scrapping article n°248
Scrapping article n°249
Scrapping article n°250
Scrapping article n°251
Scrapping article n°252
Scrapping article n°253
Scrapping article n°254
Scrapping article n°255
Scrapping article n°256
Scrapping article n°257
Scrapping article n°258
Scrapping article n°259
Scrapping article n°260
Scrapping article n°261
Scrapping article n°262
Scrapping article n°263
Scrapping article n°264
Scrapping article n°265
Scrapping article n°266
Scrapping article n°267
Scrapping article n°268
Scrapping article n°269
Scrapping article n°270
Scrapping article n°271
Scrapping article n°272
Scrapping article n°273
Scrapping article n°274
Scrapping article n°275
Scrapping article n°276
Scrapping article n°277
Scrapping article n°278
Scrapping article n°279
Scrapping article n°280
Scrapping article n°281
Scrapping article n°282
Scrapping articl

Scrapping article n°112
Scrapping article n°113
Scrapping article n°114
Scrapping article n°115
Scrapping article n°116
Scrapping article n°117
Scrapping article n°118
Scrapping article n°119
Scrapping article n°120
Scrapping article n°121
Scrapping article n°122
Scrapping article n°123
Scrapping article n°124
Scrapping article n°125
Scrapping article n°126
Scrapping article n°127
Scrapping article n°128
Scrapping article n°129
Scrapping article n°130
Scrapping article n°131
Scrapping article n°132
Scrapping article n°133
Scrapping article n°134
Scrapping article n°135
Scrapping article n°136
Scrapping article n°137
Scrapping article n°138
Scrapping article n°139
Scrapping article n°140
Scrapping article n°141
Scrapping article n°142
Scrapping article n°143
Scrapping article n°144
Scrapping article n°145
Scrapping article n°146
Scrapping article n°147
Scrapping article n°148
Scrapping article n°149
Scrapping article n°150
Scrapping article n°151
Scrapping article n°152
Scrapping articl

Scrapping article n°458
Scrapping article n°459
Scrapping article n°460
Scrapping article n°461
Scrapping article n°462
Scrapping article n°463
Scrapping article n°464
Scrapping article n°465
Scrapping article n°466
Scrapping article n°467
Scrapping article n°468
Scrapping article n°469
Scrapping article n°470
Scrapping article n°471
Scrapping article n°472
Scrapping article n°473
Scrapping article n°474
Scrapping article n°475
Scrapping article n°476
Scrapping article n°477
Scrapping article n°478
Scrapping article n°479
Scrapping article n°480
Scrapping article n°481
Scrapping article n°482
Scrapping article n°483
Scrapping article n°484
Scrapping article n°485
Scrapping article n°486
Scrapping article n°487
Scrapping article n°488
Scrapping article n°489
Scrapping article n°490
Scrapping article n°491
Scrapping article n°492
Scrapping article n°493
Scrapping article n°494
Scrapping article n°495
Scrapping article n°496
Scrapping article n°497
Scrapping article n°498
Scrapping articl

In [7]:
# concatenating all the dataframe of the retrieved list of dataset

df_temp = pd.DataFrame(columns=["Tag", "year","month", "claps", "respond", "title", "authors", "link"], index=range(0))

for l in df_list:
    df_list_concat = pd.concat([df_temp, l], ignore_index=True)
    df_temp = df_list_concat

df_list_concat

,Tag,year,month,claps,respond,title,authors,link
0,conversational-ai,2024,1,30,0,New Features Agent Ai Chat Bot By I Need Leads...,Ineedleadsuk,https://medium.com/@ineedleadsuk/new-features-...
1,conversational-ai,2024,1,52,0,Pheme: An Efficient and Conversational Speech ...,Pankaj Trivedi,https://medium.com/@reachpankaj.trivedi/pheme-...
2,conversational-ai,2024,1,37,0,How to Transform Frontline Recruitment With an...,Juan Ben,https://medium.com/cloudapper-ai-for-ukg/how-t...
3,conversational-ai,2024,1,30,0,How to Handle High-Volume Frontline Hiring Wit...,Juan Ben,https://medium.com/cloudapper-ai-for-ukg/how-t...
4,conversational-ai,2024,1,435,0,MultiHop-RAG,Cobus Greyling,https://medium.com/@cobusgreyling/multihop-rag...
...,...,...,...,...,...,...,...,...
15942,sql,2024,2,0,0,SQL Data Cleaning for Handling Missing Values ...,SQL Fundamentals,https://medium.com/@sqlfundamentals/sql-data-c...
15943,sql,2024,2,0,0,Generating SQL Create Table from Pandas DataFrame,OmniView.,https://medium.com/@omniview/as-data-enthusias...
15944,sql,2024,2,0,0,[New Recipe] Entity Relationship Diagram (ERD),J.Dechavee,https://medium.com/@dechavee.j/new-recipe-enti...
15945,sql,2024,2,20,0,Calculating Running Total In SQL,Matt Damberg,https://medium.com/@mattdamberg/calculating-ru...


In [8]:
# saving the dataset into a csv and xlsx

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y-%m-%d_%Hh%M")
print("date and time =", dt_string)

df_list_concat.to_csv(f"{dt_string}_scrap global.csv", index=False) 
#df_list_concat.to_excel(f"{dt_string}_scrap global.xlsx") # error with xlsx and nan

now = 2024-03-11 09:30:42.516538
date and time = 2024-03-11_09h30


IllegalCharacterError: [Bigdata Cluster] Building a big data cluster consisting of two name nodes and three data nodes cannot be used in worksheets.

In [ ]:
break

In [ ]:
## scrapping article by article using the list of article urls scrapped above

In [94]:
# if scrapping in multiple sessions use this
last_detail_concat = pd.read_csv("2024-03-13_17h42_detailed scrap.csv", index_col=0)
df_temp = last_detail_concat
df_temp

,date,categories,title,Claps,Respond,authors,catch_phrase,text,link
0,NaN,"Ai Chat Bot, Ai Chatbot, Conversational AI, Bu...",New Features Agent Ai Chat Bot By I Need Leads...,30.0,NaN,NaN,Sign up to discover human stories that deepen ...,"In this article, I will show you some of the ...",https://medium.com/@ineedleadsuk/new-features-...
1,3,"Artificial Intelligence, Conversational AI, Vo...",Pheme: An Efficient and Conversational Speech ...,52.0,3.0,NaN,Conversation models to the next level,Pheme is a series of efficient and conversati...,https://medium.com/@reachpankaj.trivedi/pheme-...
2,"Jan 31, 2024","Frontline Worker, Workforce Management, Ai In ...",How to Transform Frontline Recruitment With an...,37.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,Think about it — who knows your company cultu...,https://medium.com/cloudapper-ai-for-ukg/how-t...
3,"Jan 31, 2024","Talent Acquisition, Workforce Management, Conv...",How to Handle High-Volume Frontline Hiring Wit...,30.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,"So, why is there a constant need for an influ...",https://medium.com/cloudapper-ai-for-ukg/how-t...
4,3,"AI, Artificial Intelligence, Machine Learning,...",MultiHop-RAG,435.0,3.0,NaN,A recent direction in RAG architecture is esta...,Considering the development of the Large Lang...,https://medium.com/@cobusgreyling/multihop-rag...
...,...,...,...,...,...,...,...,...,...
4,NaN,NaN,Advanced SQL: Window Functions,12.0,NaN,NaN,Common Use Cases:,"Window functions, a complex yet powerful feat...",https://medium.com/@product.mgmt.blog/advanced...
5,1,"Java, Data Processing, Database, Sql",What to do if the query calculation is moved o...,108.0,1.0,NaN,Sign up to discover human stories that deepen ...,"Normally, as a compiled language, Java may no...",https://medium.com/@wdn0612/what-to-do-if-the-...
6,"Jan 14, 2024","Sql, Sql Server, Postgres, Postgresql, Database",SQL — Cursors and Their Usage in Databases,109.0,NaN,Ihor Lukianov,An introduction to working with the cursor in ...,"However, SQL is a declarative language and ha...",https://medium.com/stackademic/sql-cursors-and...
7,NaN,"Data Engineering, Spark, Sql, Apache Spark, Bi...",Demystifying the Spark Shuffle,NaN,NaN,NaN,Sign up to discover human stories that deepen ...,"For example, say you have data partitioned by...",https://medium.com/@chopadahenil/demystifying-...


In [95]:
df_temp.columns

Index(['date', 'categories', 'title', 'Claps', 'Respond', 'authors',
       'catch_phrase', 'text', 'link'],
      dtype='object')

In [96]:
# as the list is huge, we scrap article 10 by 10 to make sure we can stop the scraping at some point and not lose a lot of article
# because scraping is super long = up to 3-4 days
import numpy as np
indices = np.arange(len(last_detail_concat)+1, len(df_list_concat),10)
indices

array([15174, 15184, 15194, 15204, 15214, 15224, 15234, 15244, 15254,
       15264, 15274, 15284, 15294, 15304, 15314, 15324, 15334, 15344,
       15354, 15364, 15374, 15384, 15394, 15404, 15414, 15424, 15434,
       15444, 15454, 15464, 15474, 15484, 15494, 15504, 15514, 15524,
       15534, 15544, 15554, 15564, 15574, 15584, 15594, 15604, 15614,
       15624, 15634, 15644, 15654, 15664, 15674, 15684, 15694, 15704,
       15714, 15724, 15734, 15744, 15754, 15764, 15774, 15784, 15794,
       15804, 15814, 15824, 15834, 15844, 15854, 15864, 15874, 15884,
       15894, 15904, 15914, 15924, 15934, 15944])

In [97]:
# as the list is huge, we scrap article 10 by 10 to make sure we can stop the scraping at some point and not lose a lot of article
# because scraping is super long = up to 3-4 days

for i in range(len(indices[:-1]-1)):
    print(indices[i], indices[i+1])
print(len(df_list_concat))

15174 15184
15184 15194
15194 15204
15204 15214
15214 15224
15224 15234
15234 15244
15244 15254
15254 15264
15264 15274
15274 15284
15284 15294
15294 15304
15304 15314
15314 15324
15324 15334
15334 15344
15344 15354
15354 15364
15364 15374
15374 15384
15384 15394
15394 15404
15404 15414
15414 15424
15424 15434
15434 15444
15444 15454
15454 15464
15464 15474
15474 15484
15484 15494
15494 15504
15504 15514
15514 15524
15524 15534
15534 15544
15544 15554
15554 15564
15564 15574
15574 15584
15584 15594
15594 15604
15604 15614
15614 15624
15624 15634
15634 15644
15644 15654
15654 15664
15664 15674
15674 15684
15684 15694
15694 15704
15704 15714
15714 15724
15724 15734
15734 15744
15744 15754
15754 15764
15764 15774
15774 15784
15784 15794
15794 15804
15804 15814
15814 15824
15824 15834
15834 15844
15844 15854
15854 15864
15864 15874
15874 15884
15884 15894
15894 15904
15904 15914
15914 15924
15924 15934
15934 15944
15947


In [98]:
df_temp = pd.DataFrame(columns=['date', 'categories', 'title', 'Claps', 'Respond', 'authors',
       'catch_phrase', 'text', 'link'], index=range(0))

df_temp = pd.concat([df_temp, last_detail_concat], ignore_index=True)
df_temp

C:\Users\romai\AppData\Local\Temp\ipykernel_9912\1744626634.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp = pd.concat([df_temp, last_detail_concat], ignore_index=True)


,date,categories,title,Claps,Respond,authors,catch_phrase,text,link
0,NaN,"Ai Chat Bot, Ai Chatbot, Conversational AI, Bu...",New Features Agent Ai Chat Bot By I Need Leads...,30.0,NaN,NaN,Sign up to discover human stories that deepen ...,"In this article, I will show you some of the ...",https://medium.com/@ineedleadsuk/new-features-...
1,3,"Artificial Intelligence, Conversational AI, Vo...",Pheme: An Efficient and Conversational Speech ...,52.0,3.0,NaN,Conversation models to the next level,Pheme is a series of efficient and conversati...,https://medium.com/@reachpankaj.trivedi/pheme-...
2,"Jan 31, 2024","Frontline Worker, Workforce Management, Ai In ...",How to Transform Frontline Recruitment With an...,37.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,Think about it — who knows your company cultu...,https://medium.com/cloudapper-ai-for-ukg/how-t...
3,"Jan 31, 2024","Talent Acquisition, Workforce Management, Conv...",How to Handle High-Volume Frontline Hiring Wit...,30.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,"So, why is there a constant need for an influ...",https://medium.com/cloudapper-ai-for-ukg/how-t...
4,3,"AI, Artificial Intelligence, Machine Learning,...",MultiHop-RAG,435.0,3.0,NaN,A recent direction in RAG architecture is esta...,Considering the development of the Large Lang...,https://medium.com/@cobusgreyling/multihop-rag...
...,...,...,...,...,...,...,...,...,...
15168,NaN,NaN,Advanced SQL: Window Functions,12.0,NaN,NaN,Common Use Cases:,"Window functions, a complex yet powerful feat...",https://medium.com/@product.mgmt.blog/advanced...
15169,1,"Java, Data Processing, Database, Sql",What to do if the query calculation is moved o...,108.0,1.0,NaN,Sign up to discover human stories that deepen ...,"Normally, as a compiled language, Java may no...",https://medium.com/@wdn0612/what-to-do-if-the-...
15170,"Jan 14, 2024","Sql, Sql Server, Postgres, Postgresql, Database",SQL — Cursors and Their Usage in Databases,109.0,NaN,Ihor Lukianov,An introduction to working with the cursor in ...,"However, SQL is a declarative language and ha...",https://medium.com/stackademic/sql-cursors-and...
15171,NaN,"Data Engineering, Spark, Sql, Apache Spark, Bi...",Demystifying the Spark Shuffle,NaN,NaN,NaN,Sign up to discover human stories that deepen ...,"For example, say you have data partitioned by...",https://medium.com/@chopadahenil/demystifying-...


In [99]:
# scraping the details of each article

for i in range(len(indices[:-1])):
    print(indices[i], indices[i+1])
    df_details_loop = get_details(df_list_concat[indices[i]:indices[i+1]],test=False)
    df_temp = pd.concat([df_temp, df_details_loop])


# then scrapping the last rows
df_details_loop = get_details(df_list_concat[indices[-1:]:],test=False)
df_temp = pd.concat([df_temp, df_details_loop])

15174 15184
Number of articles to be scrapped:
10
Beginning...
Scrapping article n°1
https://medium.com/@dmitry.turchenkov/bigquery-utilizing-external-tables-and-hive-partitioning-for-handling-appsflyer-data-882c448ba059?source=tag_archive---------438----------------------------
Scrapping article n°2
https://medium.com/@osama.fathi.shaheen/when-was-the-golden-age-of-video-games-d5d11458087e?source=tag_archive---------439----------------------------
Scrapping article n°3
https://medium.com/@anujhsrsaini/learn-sql-by-practice-hard-level-part-15-db7f0dfc2808?source=tag_archive---------440----------------------------
Scrapping article n°4
https://medium.com/stackademic/concept-of-pivoting-unpivoting-967ee12a5969?source=tag_archive---------441----------------------------
Scrapping article n°5
https://medium.com/@nile.bits/crud-in-sql-unleashing-the-power-of-seamless-data-manipulation-8ef5eca0b74c?source=tag_archive---------442----------------------------
Scrapping article n°6
https://medium

C:\Users\romai\AppData\Local\Temp\ipykernel_9912\520204128.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_temp = pd.concat([df_temp, df_details_loop])


15184 15194
Number of articles to be scrapped:
10
Beginning...
Scrapping article n°1
https://medium.com/@oluwafisayomibalogun/sql-murder-mystery-game-e1a6cd39e4e0?source=tag_archive---------448----------------------------
Scrapping article n°2
https://medium.com/@bennettmo16/navigating-data-challenges-in-your-career-in-sql-1fa9885e0417?source=tag_archive---------449----------------------------
Scrapping article n°3
https://medium.com/@mail2asimmanna/data-analyst-sql-questions-asked-in-swiggy-interview-dbe07b5e451f?source=tag_archive---------450----------------------------
Scrapping article n°4
https://medium.com/@rinorhajrizi1718/using-repeatable-read-isolation-level-to-prevent-read-skew-c03c98fa3fbb?source=tag_archive---------451----------------------------
Scrapping article n°5
https://medium.com/@amandeepbindra.com/top-40-database-management-system-interview-questions-answers-dbms-mcq-crack-interviews-cb8e6a20b09f?source=tag_archive---------452----------------------------
Scrapping 

Scrapping article n°5
https://medium.com/@ppkgtmm/structured-query-language-part-4-f77b4f6a3000?source=tag_archive---------22----------------------------
Scrapping article n°6
https://medium.com/@ppkgtmm/structured-query-language-part-3-70f94c1025bb?source=tag_archive---------23----------------------------
Scrapping article n°7
https://medium.com/@chwordal/data-analysis-portfolio-project-using-sql-and-tableau-nyc-tree-census-and-income-project-e5365593b5d9?source=tag_archive---------24----------------------------
Scrapping article n°8
https://medium.com/@singhaldhruv/unleash-the-speed-force-turbocharging-your-stored-procedures-%EF%B8%8F-f4d3d36ecc10?source=tag_archive---------25----------------------------
Scrapping article n°9
https://medium.com/@jessyvictany/5-things-to-consider-while-choosing-a-service-provider-for-sql-server-management-ef62d8c9d111?source=tag_archive---------26----------------------------
Scrapping article n°10
https://medium.com/@nilden84/bigqueryde-sorgu-performa

Scrapping article n°8
https://medium.com/@helgardh/gql-fa%C3%A7a-consultas-sql-no-seu-reposit%C3%B3rio-git-7bff5bdfecf6?source=tag_archive---------65----------------------------
Scrapping article n°9
https://medium.com/@nigarali/how-to-merge-dataframes-in-pandas-a8689c77de8f?source=tag_archive---------66----------------------------
Scrapping article n°10
https://medium.com/@jjmejia_dev/improving-sql-search-statements-using-php-01cadfe53915?source=tag_archive---------67----------------------------
end of get_details()
15274 15284
Number of articles to be scrapped:
10
Beginning...
Scrapping article n°1
https://medium.com/gitconnected/gcp-extracting-data-in-cloudsql-to-cloud-storage-5e4459fc377c?source=tag_archive---------68----------------------------
Scrapping article n°2
https://medium.com/@deepaksharma2494/simplify-your-data-analysis-workflow-20-must-know-sql-functions-ef25b5f42b80?source=tag_archive---------69----------------------------
Scrapping article n°3
https://medium.com/@jain

...

Scrapping article n°6
https://medium.com/code-like-a-girl/showcase-your-sql-python-and-model-evaluation-skills-with-just-few-lines-of-codes-0c22df18b2dc?source=tag_archive---------593----------------------------
Scrapping article n°7
https://medium.com/@DataStringer/data-portfolio-project-1-sql-and-power-bi-part-3-connecting-power-bi-to-sql-server-and-fe8b2ec738ed?source=tag_archive---------594----------------------------
Scrapping article n°8
https://medium.com/@bartoszpudlo07/leetcode-lets-start-with-sql-50-a621d4b6de2e?source=tag_archive---------595----------------------------
Scrapping article n°9
https://medium.com/django-unleashed/django-queryset-api-reference-alternative-sql-queries-for-each-method-457a916f244c?source=tag_archive---------596----------------------------
Scrapping article n°10
https://medium.com/@soares.mota.ch/mer-modelo-entidade-relacionamento-e-der-diagrama-entidade-relacionamento-6a5fc16ce7c6?source=tag_archive---------597----------------------------
end of ge

Scrapping article n°9
https://medium.com/@nirbhaysingh281/oracle-database-is-down-how-to-start-3fbd4d7d4118?source=tag_archive---------636----------------------------
Scrapping article n°10
https://medium.com/@fzkarami/exploring-3-often-overlooked-t-sql-functions-b17e29c5be70?source=tag_archive---------637----------------------------
end of get_details()
15844 15854
Number of articles to be scrapped:
10
Beginning...
Scrapping article n°1
https://medium.com/art-of-data-engineering/if-i-were-to-learn-sql-from-scratch-how-would-i-start-68b3d79dad2a?source=tag_archive---------638----------------------------
Scrapping article n°2
https://medium.com/@sreenivasmaddu/related-vs-relatedtable-dax-function-in-power-bi-19ce390a9df9?source=tag_archive---------639----------------------------
Scrapping article n°3
https://medium.com/@amol.deshmukh_97340/safe-data-fix-pattern-for-rdbms-117d7fc2eb00?source=tag_archive---------640----------------------------
Scrapping article n°4
https://medium.com/@gor

Scrapping article n°4
https://medium.com/@zarmeenlakhani/why-to-shift-from-excel-to-sql-fa116b25027e?source=tag_archive---------681----------------------------
Scrapping article n°5
https://medium.com/@kombib/sqlgpt-jedno-pitanje-i-dva-odgovora-d95a685e7339?source=tag_archive---------682----------------------------
Scrapping article n°6
https://medium.com/@thomas.ellyatt/a-guide-to-search-indexes-in-bigquery-3d2b586e10fb?source=tag_archive---------683----------------------------
Scrapping article n°7
https://medium.com/towardsdev/join-bigquery-table-with-spark-stream-to-get-mapping-data-in-realtime-b1a1e1b00763?source=tag_archive---------684----------------------------
Scrapping article n°8
https://medium.com/@sunansamudro/types-of-join-in-sql-8219fd91f2ae?source=tag_archive---------685----------------------------
Scrapping article n°9
https://medium.com/@nabanitaborkakati.riki/577-employee-bonus-a6eda4118f53?source=tag_archive---------686----------------------------
Scrapping article 

Scrapping article n°9
https://medium.com/codenx/choosing-sql-or-nosql-database-in-microservices-7d3047150be2?source=tag_archive---------726----------------------------
Scrapping article n°10
https://medium.com/@premvishnoi/de-dbt-what-should-you-know-ee7271f493b0?source=tag_archive---------727----------------------------
end of get_details()
15934 15944
Number of articles to be scrapped:
10
Beginning...
Scrapping article n°1
https://medium.com/devops-dev/how-to-enable-remote-connections-to-windows-sql-server-using-ip-address-127b8cee2d38?source=tag_archive---------728----------------------------
Scrapping article n°2
https://medium.com/@sqlinsix/when-to-use-sql-or-nosql-b50d4a52c157?source=tag_archive---------729----------------------------
Scrapping article n°3
https://medium.com/@munjapararonak1/net-mvc-sql-questions-2589dee5b14e?source=tag_archive---------730----------------------------
Scrapping article n°4
https://medium.com/@premvishnoi/data-engineer-postgresql-best-practice-4335

TypeError: cannot do slice indexing on RangeIndex with these indexers [[15944]] of type ndarray

In [100]:
df_temp

,date,categories,title,Claps,Respond,authors,catch_phrase,text,link
0,NaN,"Ai Chat Bot, Ai Chatbot, Conversational AI, Bu...",New Features Agent Ai Chat Bot By I Need Leads...,30.0,NaN,NaN,Sign up to discover human stories that deepen ...,"In this article, I will show you some of the ...",https://medium.com/@ineedleadsuk/new-features-...
1,3,"Artificial Intelligence, Conversational AI, Vo...",Pheme: An Efficient and Conversational Speech ...,52.0,3.0,NaN,Conversation models to the next level,Pheme is a series of efficient and conversati...,https://medium.com/@reachpankaj.trivedi/pheme-...
2,"Jan 31, 2024","Frontline Worker, Workforce Management, Ai In ...",How to Transform Frontline Recruitment With an...,37.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,Think about it — who knows your company cultu...,https://medium.com/cloudapper-ai-for-ukg/how-t...
3,"Jan 31, 2024","Talent Acquisition, Workforce Management, Conv...",How to Handle High-Volume Frontline Hiring Wit...,30.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,"So, why is there a constant need for an influ...",https://medium.com/cloudapper-ai-for-ukg/how-t...
4,3,"AI, Artificial Intelligence, Machine Learning,...",MultiHop-RAG,435.0,3.0,NaN,A recent direction in RAG architecture is esta...,Considering the development of the Large Lang...,https://medium.com/@cobusgreyling/multihop-rag...
...,...,...,...,...,...,...,...,...,...
5,"Feb 1, 2024",None,Creating Databases with SQL Server: A Beginner...,55.0,NaN,,A Message from AI Mind,My previous article was about connecting to S...,https://medium.com/ai-mind-labs/creating-datab...
6,,None,Designing a Data Model for a Ride-Sharing App:...,252.0,NaN,,Entities:,First of all let’s decide how many fact and d...,https://medium.com/@vishalbarvaliya/designing-...
7,"POST _sql?format=txt\n{\n ""query"": """"""\n SEL...","Sql, Elasticsearch, Engineering",[工作筆記] ElasticSearch 的 SQL API,NaN,NaN,,Sign up to discover human stories that deepen ...,建議使用單引號 ‘ 來標示一般字串，使用雙引號 “ 來標示欄位，若沒有標示正確可能會得到 ...,https://medium.com/@tses89214/%E5%B7%A5%E4%BD%...
8,1,None,SQL Data Cleaning for Handling Missing Values ...,119.0,1.0,,Conclusion,Data cleaning is a critical step in the data ...,https://medium.com/@sqlfundamentals/sql-data-c...


In [101]:
# saving the detail dataset into a csv

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y-%m-%d_%Hh%M")
print("date and time =", dt_string)


df_temp.to_csv(f"{dt_string}_detailed scrap.csv")

now = 2024-03-13 20:26:40.266069
date and time = 2024-03-13_20h26


In [ ]:
break

In [ ]:
## in case of scraping from different computer use this to concatenate the dataframes:

In [27]:
first_detail_concat = pd.read_csv("2024-03-09_17h49_part_1_detailed scrap.csv", index_col=0)
second_detail_concat = pd.read_csv("2024-03-10_13h06_part_2_detailed scrap.csv", index_col=0)
third_detail_concat = pd.read_csv("2024-02-21_15h38_part_4_detailed scrap.csv", index_col=0)


In [28]:
df_temp2 = pd.DataFrame(columns=['date', 'categories', 'title', 'Claps', 'Respond', 'authors',
       'catch_phrase', 'text', 'link'], index=range(0))


df_temp2 = pd.concat([first_detail_concat, second_detail_concat, third_detail_concat
                     ], ignore_index=True)
df_temp2

,date,categories,title,Claps,Respond,authors,catch_phrase,text,link
0,NaN,"Ai Chat Bot, Ai Chatbot, Conversational AI, Bu...",New Features Agent Ai Chat Bot By I Need Leads...,30.0,NaN,NaN,Sign up to discover human stories that deepen ...,"In this article, I will show you some of the ...",https://medium.com/@ineedleadsuk/new-features-...
1,3,"Artificial Intelligence, Conversational AI, Vo...",Pheme: An Efficient and Conversational Speech ...,52.0,3.0,NaN,Conversation models to the next level,Pheme is a series of efficient and conversati...,https://medium.com/@reachpankaj.trivedi/pheme-...
2,"Jan 31, 2024","Frontline Worker, Workforce Management, Ai In ...",How to Transform Frontline Recruitment With an...,37.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,Think about it — who knows your company cultu...,https://medium.com/cloudapper-ai-for-ukg/how-t...
3,"Jan 31, 2024","Talent Acquisition, Workforce Management, Conv...",How to Handle High-Volume Frontline Hiring Wit...,30.0,NaN,CloudApper AI — Simplify AI/LLM Adoption for E...,Sign up to discover human stories that deepen ...,"So, why is there a constant need for an influ...",https://medium.com/cloudapper-ai-for-ukg/how-t...
4,3,"AI, Artificial Intelligence, Machine Learning,...",MultiHop-RAG,434.0,3.0,NaN,A recent direction in RAG architecture is esta...,Considering the development of the Large Lang...,https://medium.com/@cobusgreyling/multihop-rag...
...,...,...,...,...,...,...,...,...,...
1863,NaN,"Data, Sql, Web3",Tableland: Pioneering the Future of Decentrali...,3.0,NaN,NaN,Sign up to discover human stories that deepen ...,NaN,https://medium.com/@hopynovel000/tableland-pio...
1864,NaN,"Snowflake Native App, Sql, Semi Structured Dat...",A Snowflake-Native Health App,12.0,NaN,NaN,From Semi-structured Health Data to Full Stack...,The data being used in this application was c...,https://medium.com/@ericfflynn/a-snowflake-nat...
1865,NaN,"Sql, Business, Sustainability, Data Analysis, ...",Data Mart: From Green Gamble to Red Numbers,18.0,NaN,NaN,Case Study 5 of Danny Ma’s #8WeekSQLChallenge,With the ever-increasing need for sustainable...,https://medium.com/@neto_a/data-mart-from-gree...
1866,NaN,"Mssql, Sql, Data Science",Introduction to MS SQL,24.0,NaN,NaN,1. Create a New Database,2.1 Create New Table Page Click with the righ...,https://medium.com/@fatmanurkutlu1/introductio...


In [102]:
#checking the unique url = the lenght of the dataset to avoid duplicates or missing values

df_temp2["link"].unique

<bound method Series.unique of 0    https://medium.com/@ineedleadsuk/new-features-...
1    https://medium.com/@reachpankaj.trivedi/pheme-...
2    https://medium.com/cloudapper-ai-for-ukg/how-t...
3    https://medium.com/cloudapper-ai-for-ukg/how-t...
4    https://medium.com/@cobusgreyling/multihop-rag...
                           ...                        
5    https://medium.com/ai-mind-labs/creating-datab...
6    https://medium.com/@vishalbarvaliya/designing-...
7    https://medium.com/@tses89214/%E5%B7%A5%E4%BD%...
8    https://medium.com/@sqlfundamentals/sql-data-c...
9    https://medium.com/@omniview/as-data-enthusias...
Name: link, Length: 15943, dtype: object>

In [25]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y-%m-%d_%Hh%M")
print("date and time =", dt_string)


df_temp2.to_csv(f"{dt_string}_all_detailed scrap.csv")

now = 2024-02-21 15:40:11.206593
date and time = 2024-02-21_15h40
